In [4]:
### all the necessary imports

# python script imports
import sys
sys.path.insert(0, '../scripts')
from sparkshape import *

# This is copied from tutorial 1
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

# other imports
from pyspark.sql.functions import col
from pyspark.sql.functions import unix_timestamp

22/08/17 14:30:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
# let's import all the green taxi data for the year 2019 and take a closer look into what the data looks like
sdf_g = spark.read.parquet("../data/raw/green/")
sdf_g.printSchema()
sdf_g.show()

root
 |-- VendorID: long (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- trip_type: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)

+--------+--------------------+---------------------+------------------+----------+------------+-

In [3]:
# Alright! Now that I've had a good idea what this data looks like, let's remove ehail_fee out of the data 
# because the values in ehail_fee are always null. I also need data for the length of each trip, and that is acquired by calculating
# the difference between lpep_pickup_datetime and lpep_dropoff_datetime.
time_diff = unix_timestamp("lpep_dropoff_datetime") - unix_timestamp("lpep_pickup_datetime")
sdf_g = sdf_g.drop(col("ehail_fee"))
sdf_g = sdf_g.withColumn("trip_duration", time_diff)

In [4]:
# save this as a new parquet file
sdf_g.write.mode("overwrite").parquet("../data/curated/green")
# to copy what tute1 said: "Your directory might look a bit funky. Don't worry, just leave it as is,
# (we don't have time to cover everything about Spark unfortunately) and you can just read in the directory as is."
# I TAKE YOUR WORD FOR THIS, OKAY!!

22/08/10 12:12:35 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
22/08/10 12:12:35 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
22/08/10 12:12:42 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


22/08/10 12:12:42 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
22/08/10 12:12:43 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
22/08/10 12:12:43 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


Now, do the same thing (yes, same exact thing) for the yellow taxi data.

In [14]:
sdf_y = spark.read.parquet("../data/raw/yellow/2019-01.parquet")
sdf_y.printSchema()
sdf_y.show()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: integer (nullable = true)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+-----------

It seems like there is no ehail_fee unlike the green data. All is good to proceed to the next step.

In [15]:
time_diff = unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")
sdf_y = sdf_y.withColumn("trip_duration", time_diff)
sdf_y = sdf_y.select(col("tpep_pickup_datetime"), col("tpep_dropoff_datetime"), col("trip_distance"), \
    col("trip_duration"), col("PULocationID"), col("DOLocationID"))
sdf_y = sdf_y.sort(col("tpep_pickup_datetime"),col("tpep_dropoff_datetime"))

In [69]:
from datetime import datetime
sdf_y = sdf_y.where(sdf_y.tpep_dropoff_datetime.to_datetime() > datetime(2018, 12, 31)).where(sdf_y.tpep_pickup_datetime < datetime(2020, 1, 1))
sdf_y

ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/home/andrew/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/andrew/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


In [7]:
for i in range(1, 13):
    if i < 10:
        sdf_y = spark.read.parquet("../data/raw/yellow/2019-0" + str(i) + ".parquet")
        time_diff = unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")
        sdf_y = sdf_y.withColumn("trip_duration", time_diff)
        sdf_y.write.mode("overwrite").parquet("../data/curated/yellow/2019-0" + str(i))
    else:
        sdf_y = spark.read.parquet("../data/raw/yellow/2019-" + str(i) + ".parquet")
        time_diff = unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")
        sdf_y = sdf_y.withColumn("trip_duration", time_diff)
        sdf_y.write.mode("overwrite").parquet("../data/curated/yellow/2019-" + str(i))

22/08/10 14:19:14 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 904277 ms exceeds timeout 120000 ms
22/08/10 14:19:14 WARN SparkContext: Killing executors is not supported by current scheduler.


On a day-to-day basis, compare a number of attributes to the total number of crashes in that timeframe.
In one day, the rows are further broken down into: the month (to account for the average weather condition), the daylight (day or night), the most common human waking hours (11 PM to 7 AM vs the rest of the day), and rush hours [FOR WEEKDAYS AND NON-PUBLIC HOLIDAYS ONLY].

For the rush hours, the TLC defines it as a period between 4 PM to 8 PM, which is arbitrary because there should be rush hours in the morning as well. The true time period of rush hours can be obtained by measuring the average speed of all taxi trips measured on that day. However, that arbitrary definition should be good enough to use because, (1) there will be a significant increase in the amount of time and effort to analyse the average speed data day-by-day, (2) chances are the most of the period between 4 to 8 PM will overlap with the actual true period measured by taxi speed, (3) the speed difference during the day between the rush and non-rush hours might not be too big anyway, compared to the speed difference between day and night.

For the waking vs non-waking hours, it is again somewhat arbitrary, but it's pretty close to the data gathered by Sleep Cycle.
https://www.sleepcycle.com/sleep-science/what-we-know-about-americas-healthiest-happiest-best-rested/

Unfortunately, there is no data for the education status, income, age, and driving experience for taxi drivers in new york, as a significant amount of crashes are caused by human errors and disobeyment of road rules. So, we gotta make do with what we got.


1. Read twilight data, convert datetime into three columns of data: one for the month, one for day or night taken from the twilight data, one for the peak hour or non-peak hour.
2. That makes it three attributes. The fourth one is the number of trips completed per instance, from the yellow plus green taxi data.
3. On the weekends and public holidays, there will be no rush vs non-rush hours, there will just be another value in the attribute labelled "OFF" in the rush_hours column.
4. In addition, the average human waking vs sleeping hours will be included.
5. The location of the crash is probably the most complicated part. Join location and get its location ID from the crash data. (note: rtree or pygeos need to be installed)
7. Any additional attributes will be improvised along the way.

ANALYSIS
1. Definitely linear regression.
2. What else, I don't know.

attributes:   ["month, daylight, waking_hours, rush_hours, locationID, num_victims (injured + killed), num_vehicles_involved, total_num_of_taxi_trips]
response: ["total number of crashes"]


1. Read twilight data, convert datetime into three columns of data: one for the month, one for day or night taken from the twilight data, one for the peak hour or non-peak hour.
2. The rush or peak hours will be obtained by the average speed in which the taxis travel in a given time, every 10 minutes. The trend then can be seen smoothly in a graph. (maybe divide into 3 section: one for rush hours, one for normal, and one for really empty streets where vehicles can go so fast)
3. Do this on the weekends and public holidays as well.
See the trends, and decide which hours to divide.
4. In addition, the average human waking vs sleeping hours will be included.
5. The location of the crash is probably the most complicated part. Join location and get its location ID from the crash data.
7. Any additional attributes will be added and improvised along the way.

-------------------------------------------------------------------------------------------------------------------------------------------------

Okay, I'm **done** with spark. The timezone is messed up and the "timestamp" data type doesn't match properly with datetime(), so there is no way to precisely see data outside of 2019 nor there is a way to filter out specific dates to find out weekdays/weekends and public holidays, nor there is an accurate way to see day vs night.

Moving on to the ol' reliable pandas.

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime

In [5]:
pdf = pd.read_parquet("../data/raw/yellow/2019-01.parquet")  # pdf = parquet data frame

# drop useless columns
pdf = pdf.drop(columns=["VendorID", "passenger_count", "RatecodeID", "store_and_fwd_flag", "payment_type", "fare_amount", "extra", "mta_tax" ,\
    "tip_amount", "tolls_amount", "improvement_surcharge", "total_amount", "congestion_surcharge", "airport_fee"])
# filter out non-2019 data
pdf = pdf[pdf["tpep_dropoff_datetime"] > datetime(2018, 12, 31)]
pdf = pdf[pdf["tpep_pickup_datetime"] < datetime(2020, 1, 1)]
pdf.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID
0,2019-01-01 00:46:40,2019-01-01 00:53:20,1.5,151,239
1,2019-01-01 00:59:47,2019-01-01 01:18:59,2.6,239,246
7,2019-01-01 00:21:28,2019-01-01 00:28:37,1.3,163,229
8,2019-01-01 00:32:01,2019-01-01 00:45:39,3.7,229,7
9,2019-01-01 00:57:32,2019-01-01 01:09:32,2.1,141,234


In [6]:
# find out the trip duration
pdf["trip_duration"] = pdf["tpep_dropoff_datetime"] - pdf["tpep_pickup_datetime"]
# in minutes (float)
pdf["trip_duration"] = pdf["trip_duration"] / np.timedelta64(1, "m")
pdf.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,trip_duration
0,2019-01-01 00:46:40,2019-01-01 00:53:20,1.5,151,239,6.666667
1,2019-01-01 00:59:47,2019-01-01 01:18:59,2.6,239,246,19.200000
7,2019-01-01 00:21:28,2019-01-01 00:28:37,1.3,163,229,7.150000
8,2019-01-01 00:32:01,2019-01-01 00:45:39,3.7,229,7,13.633333
9,2019-01-01 00:57:32,2019-01-01 01:09:32,2.1,141,234,12.000000


## 1: Speed

In [7]:
# find out the average speed of each trip (minutes/mile)
pdf["avg_speed"] = pdf["trip_duration"] / pdf["trip_distance"]
pdf.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,trip_duration,avg_speed
0,2019-01-01 00:46:40,2019-01-01 00:53:20,1.5,151,239,6.666667,4.444444
1,2019-01-01 00:59:47,2019-01-01 01:18:59,2.6,239,246,19.200000,7.384615
7,2019-01-01 00:21:28,2019-01-01 00:28:37,1.3,163,229,7.150000,5.500000
8,2019-01-01 00:32:01,2019-01-01 00:45:39,3.7,229,7,13.633333,3.684685
9,2019-01-01 00:57:32,2019-01-01 01:09:32,2.1,141,234,12.000000,5.714286


Before proceeding any further, we must first see and assess outliers, because I *know* there will be outliers in this big messy data.

In [8]:
pdf.sort_values("avg_speed", ascending=True)
print(pdf.head())
pdf.tail()

  tpep_pickup_datetime tpep_dropoff_datetime  trip_distance  PULocationID  \
0  2019-01-01 00:46:40   2019-01-01 00:53:20            1.5           151   
1  2019-01-01 00:59:47   2019-01-01 01:18:59            2.6           239   
7  2019-01-01 00:21:28   2019-01-01 00:28:37            1.3           163   
8  2019-01-01 00:32:01   2019-01-01 00:45:39            3.7           229   
9  2019-01-01 00:57:32   2019-01-01 01:09:32            2.1           141   

   DOLocationID  trip_duration  avg_speed  
0           239       6.666667   4.444444  
1           246      19.200000   7.384615  
7           229       7.150000   5.500000  
8             7      13.633333   3.684685  
9           234      12.000000   5.714286  


,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,trip_duration,avg_speed
7696612,2019-01-31 23:37:20,2019-02-01 00:10:43,10.24,142,95,33.383333,3.260091
7696613,2019-01-31 23:28:00,2019-01-31 23:50:50,12.43,48,213,22.833333,1.836954
7696614,2019-01-31 23:11:00,2019-01-31 23:46:00,9.14,159,246,35.000000,3.829322
7696615,2019-01-31 23:03:00,2019-01-31 23:14:00,0.00,265,265,11.000000,inf
7696616,2019-01-31 23:41:03,2019-02-01 00:19:16,12.30,237,197,38.216667,3.107046


In [27]:
# the speed data for every nth minute in one day is going to be stored in a 2d list, consisting of 1440 * x elements.
# note: 1440 is the number of minutes in 24 hours, and x is the avg_speed data when (a) trip(s) occurs at the nth minute.
speed_data = [[]]*1440

for i in range(0, 3):
    # the average speed value for this trip
    avg_speed = pdf.iloc[i, -1]

    # pick up time, in the number of minutes elapsed starting from 00:00
    pu_day = pdf.iloc[i, 0].timetuple().tm_yday
    pu_hour = pdf.iloc[i, 0].timetuple().tm_hour
    pu_min = pdf.iloc[i, 0].timetuple().tm_min + 1  # rounding up, because tm_sec exists (why? explained in report, 
                                                    # if the page limit allows...otherwise, think about it yourself, it's pretty intuitive)
    pu_time = 60 * pu_hour + pu_min

    # drop off time, in the number of minutes elapsed starting from 00:00
    do_day = pdf.iloc[i, 1].timetuple().tm_yday
    do_hour = pdf.iloc[i, 1].timetuple().tm_hour
    do_min = pdf.iloc[i, 1].timetuple().tm_min
    do_time = 60 * do_hour + do_min

    
    if do_day != pu_day:
    # if the trip starts at a particular day and ends at the next day(s)
        pass
    else:
    # if the trip starts and ends at the same day
        while pu_time < do_time:
            speed_data[pu_time].append(avg_speed)
            pu_time += 1

speed_data

[[4.444444444444445,
  4.444444444444445,
  4.444444444444445,
  4.444444444444445,
  4.444444444444445,
  4.444444444444445,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5,
  5.5],
 [4.444444444444445,
  4.444444444444445,
  4.444444444444445,
  4.444444444444445,
  4.444444444444445,
  4.444444444444445,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.384615384615384,
  7.38461538